In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image, ImageDraw

Using TensorFlow backend.


In [6]:
#genomes as a list of gene sequences
genomes =[]
for i in range(2):
    genomes.append([np.random.randint(2, size=5)]) # add 5 randomly initialised indiviudals

    
print (genomes)

accuracy_of_population= []

[[array([1, 1, 0, 1, 0])], [array([1, 1, 0, 1, 0])]]


In [ ]:
for element in genomes:
    for indiv in element:
        print(indiv)
        




    batch_size = 128
    num_classes = 10
    epochs = 12

    # input image dimensions
    img_rows, img_cols = 28, 28

    # the data, split between train and test sets
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    if K.image_data_format() == 'channels_first':
        x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
        x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
        x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    model = Sequential()




    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    ########

    for gen in indiv:
        if (gen== 0):
            model.add(Conv2D(64, (3, 3), activation='relu'))
        elif (gen == 1):
            model.add(Dense(128, activation='relu'))




    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])

    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    accuracy_of_population.append(score[0])

print(accuracy_of_population)

[1 1 0 1 0]
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 36s 608us/step - loss: 0.2654 - acc: 0.9167 - val_loss: 0.0556 - val_acc: 0.9830
Epoch 2/12
60000/60000 [==============================] - 35s 588us/step - loss: 0.0733 - acc: 0.9770 - val_loss: 0.0507 - val_acc: 0.9863
Epoch 3/12
60000/60000 [==============================] - 38s 626us/step - loss: 0.0530 - acc: 0.9837 - val_loss: 0.0318 - val_acc: 0.9891
Epoch 4/12
60000/60000 [==============================] - 37s 610us/step - loss: 0.0436 - acc: 0.9863 - val_loss: 0.0274 - val_acc: 0.9900
Epoch 5/12
60000/60000 [==============================] - 36s 606us/step - loss: 0.0384 - acc: 0.9883 - val_loss: 0.0254 - val_acc: 0.9908
Epoch 6/12
60000/60000 [==============================] - 35s 586us/step - loss: 0.0331 - acc: 0.9899 - val_loss: 0.0258 - val_acc: 0.9912
Epoch 7/12
60000/60000 [=======